In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

data_path = '/content/drive/MyDrive/training'
class_names = os.listdir(data_path)
num_classes = len(class_names)

data = []
labels = []

for class_name in class_names:
    class_path = os.path.join(data_path, class_name)
    for image_name in os.listdir(class_path):
        image_path = os.path.join(class_path, image_name)
        try:
            image = cv2.imread(image_path)
            if image is None:
                raise ValueError(f"Invalid image file: {image_path}")
            image = cv2.resize(image, (224, 224))
            data.append(image)
            labels.append(class_names.index(class_name))
        except Exception as e:
            print(f"Error processing image: {image_path}")
            print(e)

data = np.array(data)
labels = np.array(labels)



Splitting the dataset into Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split

train_data, test_data, train_labels, test_labels = train_test_split(data, labels, test_size=0.2, random_state=42)


In [ ]:
train_data = train_data.astype('float32')
test_data = test_data.astype('float32')

mean = np.mean(train_data, axis=(0, 1, 2))
std = np.std(train_data, axis=(0, 1, 2))

train_data = (train_data - mean) / std
test_data = (test_data - mean) / std


Installing the required packages

In [ ]:
!pip install --upgrade tensorflow
!pip install --upgrade keras


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.8/47.8 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 601.3/601.3 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 235.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 71.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 101.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 358.9/358.9 kB 24.0 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.2.0
    Uninstalling ml-dtypes-0.2.0:
      Successfully uninstalled ml-dtypes-0.2.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.15.2
    Uninstalling tensorboard-2.15.2:
      Successfully uninstalled tensorboard-2.15.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
  Attempting uninstall: tensorflow
 

In [ ]:
from sklearn.utils import class_weight

class_weights = class_weight.compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)
class_weights = dict(enumerate(class_weights))


resnet and attention mechanism

In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization, Multiply, Conv2D, Reshape
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, LearningRateScheduler
from tensorflow.keras import regularizers
import numpy as np

# Load the pre-trained ResNet50 model without the top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Attention mechanism (Channel Attention)
def attention_module(input_tensor):
    filters = input_tensor.shape[-1]
    gap = GlobalAveragePooling2D()(input_tensor)
    dense_1 = Dense(filters // 8, activation='relu')(gap)
    dense_2 = Dense(filters, activation='sigmoid')(dense_1)
    scale = Reshape((1, 1, filters))(dense_2)
    return Multiply()([input_tensor, scale])

# Add custom layers on top of the base model
x = base_model.output
x = attention_module(x)  # Apply the attention mechanism
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu', kernel_regularizer=regularizers.l2(0.001))(x)  # L2 Regularization
x = BatchNormalization()(x)  # Add Batch Normalization
x = Dropout(0.5)(x)  # Dropout to reduce overfitting
predictions = Dense(num_classes, activation='softmax')(x)  # Softmax activation for multi-class classification

# Define the complete model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze the base model layers for initial training
for layer in base_model.layers:
    layer.trainable = False

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Learning rate scheduler function
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return float(lr * tf.math.exp(-0.1))

# Callbacks for learning rate reduction, early stopping, and learning rate scheduling
lr_reduction = ReduceLROnPlateau(monitor='val_loss', patience=3, verbose=1, factor=0.2, min_lr=1e-5)
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
lr_scheduler = LearningRateScheduler(scheduler)

# Train the model
history = model.fit(
    train_data,
    train_labels,
    epochs=20,
    batch_size=32,
    validation_data=(test_data, test_labels),
    callbacks=[lr_reduction, early_stopping, lr_scheduler]
)

# Fine-tuning: Unfreeze base model layers and continue training
for layer in base_model.layers:
    layer.trainable = True

# Recompile the model with a lower learning rate for fine-tuning
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-5), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Continue training (fine-tuning)
history_fine_tune = model.fit(
    train_data,
    train_labels,
    epochs=10,
    batch_size=32,
    validation_data=(test_data, test_labels),
    callbacks=[lr_reduction, early_stopping, lr_scheduler]
)

# Evaluate the model on validation data
loss, accuracy = model.evaluate(test_data, test_labels)
print("Test Loss:", loss)
print("Test Accuracy:", accuracy)

# Save the model for later use
model.save('/content/retinal_disease_model_resnet_attention.h5')


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
Epoch 1/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 161s 609ms/step - accuracy: 0.6142 - loss: 2.0881 - val_accuracy: 0.6927 - val_loss: 1.2869 - learning_rate: 0.0010
Epoch 2/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 150s 595ms/step - accuracy: 0.7398 - loss: 1.1337 - val_accuracy: 0.7608 - val_loss: 0.9647 - learning_rate: 0.0010
Epoch 3/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 149s 593ms/step - accuracy: 0.7803 - loss: 0.9133 - val_accuracy: 0.7225 - val_loss: 1.0114 - learning_rate: 0.0010
Epoch 4/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 151s 601ms/step - accuracy: 0.8023 - loss: 0.7708 - val_accuracy: 0.7747 - val_loss: 0.8329 - learning_rate: 0.0010
Epoch 5/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 152s 601ms/step - accuracy: 0.8261 - loss: 0.7044 - val_accuracy: 0.6027 - val_loss: 1.5574 - learning_rate: 0.0010
Epoch 6/20
252/252 ━━━━━━━━━━━━━━━━━━━━ 151s 598ms/step - accuracy: 0.8324 - loss: 0.6600 - val_accuracy: 0.6718 - val_loss: 1.1530 - learning_rate: 0.0010
Epoch 7/20
25